# Load the LLM

The first step would be to create a method able to receive a prompt a generate a response given the model name. Please, make sure you have downloaded the model before using `ollama pull`.

**Note**: The first time you use this method, it will take some seconds to process a prompt.

In [7]:
import ollama


def get_completion(prompt:str, model_name:str) -> str:
    """
    Generate a response from the LLM model given a prompt.

    Args:
        prompt (str): The input prompt to the model.
        model_name (str): The name of the model to use.

    Returns:
        str: The response generated by the model.
    """
    response = ollama.chat(model=model_name, messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"]

# Example usage
response = get_completion("What is the capital of France?", "qwen2.5:3b")
print(f"response: {response}")
# Example usage - Create a more complex prompt
style = """American English \
in a calm and respectful tone.    
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
prompt = f"""
Translate the text \
taht is delimited by triple backticks \
into a style that is {style}. \
text: ```{customer_email}```. Please, remove any backticks in your response.
"""

response = get_completion(prompt, "qwen2.5:3b")
print(f"response: {response}")

response: The capital of France is Paris.
response: I am really frustrated because my blender lid popped off and splattered the smoothie all over my kitchen walls! And it gets worse - the warranty doesn't cover the cost of cleaning up the mess. I need your help right now, matey!


# Use of LangChain

Now, we are going to try something similar, but using LangChain. The first step is to create the chat model. To do so, we have to use the object `ChatOllama`.

In [8]:
from langchain_community.chat_models import ChatOllama

chat = ChatOllama(model="qwen2.5:3b", temperature=0.0)
chat

ChatOllama(model='qwen2.5:3b', temperature=0.0)

Then, we create a template string. As we can see, it's not going to be a formatted string.

In [9]:
template_string = """Translate the text \
    that is delimited by triple backticks \
    into a style that is {style}. \
    text: ```{text}```"""

The next step is to create the prompt template object from langchain, using the previous prompt. This is done using `ChatPromptTemplate`.

In [10]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text     that is delimited by triple backticks     into a style that is {style}.     text: ```{text}```')

We can observe that the prompt object has two **input variables: style and text**. We are going to define these variables and create the final input message for the chat:

In [11]:
customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

custom_messages = prompt_template.format_messages(style=customer_style, text=customer_email)
print(type(custom_messages[0]))

<class 'langchain_core.messages.human.HumanMessage'>


Finally, we call the model using the custom message we have created before:

In [12]:
response = chat(custom_messages)
print(response)
print(type(response))

content='```\nI am quite frustrated because my blender lid popped off and sprayed my kitchen walls with smoothie all over them. And unfortunately, the warranty does not cover the expense of cleaning up the mess in my kitchen. I need your assistance urgently, fellow pirate.\n```' additional_kwargs={} response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-10-18T14:43:36.1918589Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1214117800, 'load_duration': 23594500, 'prompt_eval_count': 113, 'prompt_eval_duration': 36000000, 'eval_count': 54, 'eval_duration': 1150000000} id='run--8536cbed-697e-4223-a96e-5e004762007e-0'
<class 'langchain_core.messages.ai.AIMessage'>


We can observe that the returned variable by the LLM is not a plain text. It corresponds to an `AIMessage`, a type of message in LangChain, usually generated by a LLM model. In order to convert these type of messages to desired format, we need an output parser.

In [13]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

# Create the prompt template object from the review template
prompt_template = ChatPromptTemplate.from_template(review_template)
# Create the messages for the chat model
messages = prompt_template.format_messages(text=customer_review)
# Call the chat model with the messages
response = chat(messages)
print(response)
print(type(response))

content='```json\n{\n  "gift": True,\n  "delivery_days": 2,\n  "price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```' additional_kwargs={} response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-10-18T14:43:39.6153551Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1367453400, 'load_duration': 47759700, 'prompt_eval_count': 240, 'prompt_eval_duration': 137000000, 'eval_count': 53, 'eval_duration': 1179000000} id='run--9f1c2ed1-77a5-4b85-8fba-0ce63a0be529-0'
<class 'langchain_core.messages.ai.AIMessage'>


As we can see, the model returned a JSON format response, since we required it in the prompt. We are going to parse this output into a Python dictionary, since it's the required format.

In [26]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# We create a schema for each of the expected outputs keys
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


The variable `format_conclusions` is a string with orders about the output format of the model must be.

In [27]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

response = chat(messages)
output_dict = output_parser.parse(response.content)
print(output_dict)

{'gift': 'True', 'delivery_days': '2', 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


As we can see, we got a dict type response, instead of AIMessage object with plain text in it's `content` argument.